COLLECTE DONNEES ENTREPRISES

In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Configuration du WebDriver
driver = webdriver.Chrome()
url = "https://www.trustpilot.com/categories/atm"
driver.get(url)
wait = WebDriverWait(driver, 10)
time.sleep(2)

# Fermer le bouton cookie
try:
    button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler')))
    button.click()
    print("Bouton cliqué avec succès.")
except Exception as e:
    print("Impossible de cliquer sur le bouton :", e)

# Collecter les informations sur les entreprises
entreprise_elements = driver.find_elements(By.CLASS_NAME, 'styles_businessUnitMain__PuwB7')

informations_entreprises = []

for entreprise in entreprise_elements:
    try:
        nom_entreprise = entreprise.find_element(By.CSS_SELECTOR, 'p.styles_displayName__GOhL2').text
    except NoSuchElementException:
        nom_entreprise = 'NA'
        
    try:
        trust_score_element = entreprise.find_element(By.CSS_SELECTOR, 'span.typography_body-m__xgxZ_')
        trust_score = trust_score_element.text.split()[-1]
    except NoSuchElementException:
        trust_score = 'NA'
        
    try:
        reviews = int(entreprise.find_element(By.CSS_SELECTOR, 'p.styles_ratingText__yQ5S7').text.split('|')[-1].strip().replace(',', '').split()[0])
    except NoSuchElementException:
        reviews = 'NA'
        
    try:
        emplacement = entreprise.find_element(By.CSS_SELECTOR, 'span.styles_location__ILZb0').text
    except NoSuchElementException:
        emplacement = 'NA'

    if reviews == 'NA':
        trust_score = 'NA'

    informations_entreprises.append({
        'Nom': nom_entreprise,
        'TrustScore': trust_score,
        'Reviews': reviews,
        'Emplacement': emplacement,
    })

# Créer un DataFrame avec les informations des entreprises
df_entreprises = pd.DataFrame(informations_entreprises)

# Ajouter des informations supplémentaires
div_elements = driver.find_elements(By.XPATH, '//div[@class="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2"]')
result_list = []
for div in div_elements:
    elements = div.find_element(By.CLASS_NAME, 'styles_footer__DoJVj')
    elements_2 = elements.find_element(By.CLASS_NAME, 'styles_wrapper___E6__')
    span_elements = elements_2.find_elements(By.TAG_NAME, 'span')
    concatenated_text = '' 
    for span in span_elements:
        concatenated_text += span.text 

    result_list.append(concatenated_text)

df_new_column = pd.DataFrame(result_list, columns=['infos'])
df_entreprises = df_entreprises.assign(infos=df_new_column['infos'])

# Extraire des informations supplémentaires et les ajouter au DataFrame
df_entreprises['review'] = df_entreprises['Reviews'].apply(lambda x: int(str(x).replace(',', '')) if x != 'NA' else 0)
df_entreprises['town'] = df_entreprises['Emplacement'].apply(lambda x: str(x).split(',')[0])
df_entreprises['country'] = df_entreprises['Emplacement'].apply(lambda x: str(x).split(',')[-1])
df_entreprises['institution_type'] = df_entreprises['infos'].apply(lambda x: str(x).split('·')[0])
df_entreprises['five_star_%'] = None  # Placeholder for the next part
df_entreprises['company_name'] = df_entreprises['Nom']
df_entreprises['trust_score'] = df_entreprises['TrustScore']

df_entreprises = df_entreprises.drop(columns=['Nom', 'TrustScore', 'Reviews', 'Emplacement', 'infos'])

# Récupérer les liens des entreprises
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)
liens = []

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    liens.append(href)

# Collecter le score 5 étoiles pour chaque entreprise
scores_5_etoiles = []

for href in liens:
    driver.execute_script("window.open('{}', '_blank');".format(href))
    driver.switch_to.window(driver.window_handles[-1])
    
    try:
        score_5_etoile = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div/div[4]/section/div[2]/div[2]/label[1]/p[2]').text
    except NoSuchElementException:
        score_5_etoile = 'NA'
    
    scores_5_etoiles.append(score_5_etoile)
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

# Ajouter les scores 5 étoiles au DataFrame
df_entreprises['five_star_%'] = scores_5_etoiles

# Sauvegarder le DataFrame
df_entreprises.to_csv(r'C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\informations_entreprises.csv', index=False)

driver.quit()


Bouton cliqué avec succès.


COLLECTE DONNEES COMMENTAIRES

In [7]:
import pandas as pd
from selenium import webdriver
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Recharger les informations des entreprises
df_entreprises = pd.read_csv('informations_entreprises.csv')

# Configuration du WebDriver
driver = webdriver.Chrome()
driver.get("https://www.trustpilot.com/categories/atm")
wait = WebDriverWait(driver, 10)

# Fermer le bouton cookie
try:
    button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler')))
    button.click()
    print("Bouton cliqué avec succès.")
except Exception as e:
    print("Impossible de cliquer sur le bouton :", e)

# Récupérer les liens des entreprises
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.paper_paper__1PY90.paper_outline__lwsUX.card_card__lQWDv.card_noPadding__D8PcU.styles_wrapper__2JOo2'))
)
liens = []

for element in elements:
    link = element.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    liens.append(href)

# Collecter les commentaires pour chaque entreprise
commentaires_par_agence = []

for href in liens:
    driver.execute_script("window.open('{}', '_blank');".format(href))
    driver.switch_to.window(driver.window_handles[-1])

    titre = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/main/div/div[3]/div[2]/div/div/div/section/div[1]/div[2]/h1/span[1]'))
    )
    titre_text = titre.text

    comments = driver.find_elements(By.CSS_SELECTOR, 'article.styles_reviewCard__hcAvl')
    i = 0

    for comment in comments:
        try:
            user_name = comment.find_element(By.CSS_SELECTOR, '.typography_heading-xxs__K8PzI').text
        except NoSuchElementException:
            user_name = 'NA' 
        try:
            location_element = driver.find_elements(By.CSS_SELECTOR, '[data-consumer-country-typography] > span')
            location_element = location_element[i]
            user_location = location_element.text
        except NoSuchElementException:
            user_location = 'NA' 
        try:
            review_content_element = driver.find_element(By.CSS_SELECTOR, 'div[data-review-content]')
            review_content = review_content_element.text
        except NoSuchElementException:
            review_content = 'NA' 
        try:
            title = comment.find_element(By.CSS_SELECTOR, '.typography_heading-s__f7029').text
        except NoSuchElementException:
            title = 'NA'          
        try:
            review_text = comment.find_element(By.CSS_SELECTOR, '.typography_body-l__KUYFJ').text
        except NoSuchElementException:
            review_text = 'NA'          
        try:
            review_count_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.styles_consumerExtraDetails__fxS4S > span.typography_body-m__xgxZ_'))
            )
            review_count = int(review_count_element.text.split()[0].replace(',', ''))
        except NoSuchElementException:
            review_count = 'NA'      
             
        try:
            date_of_experience_element = comment.find_element(By.CSS_SELECTOR, 'p[data-service-review-date-of-experience-typography]')
            date_of_experience = date_of_experience_element.text.replace('Date of experience: ', '').strip()
        except NoSuchElementException:
            date_of_experience = 'NA'
            
        try:
            reply_element = comment.find_element(By.CSS_SELECTOR, '.paper_paper__1PY90')
            reply_text = reply_element.find_element(By.CSS_SELECTOR, '.styles_message__shHhX').text
            reply_date_element = reply_element.find_element(By.CSS_SELECTOR, '.styles_replyDate__Iem0_')
            reply_date = reply_date_element.get_attribute('title')
            
            comment_data = {
                'company_name': titre_text,
                'User': user_name,
                'localisation': user_location,
                'Titre': title,
                'commentaire': review_text,
                'nombre_reviews': review_count,
                'date_experience': date_of_experience,
                'reply': {
                    'reply_text': reply_text,
                    'reply_date': reply_date
                }
            }
        except NoSuchElementException:
            comment_data = {
                'company_name': titre_text,
                'User': user_name,
                'localisation': user_location,
                'Titre': title,
                'commentaire': review_text,
                'nombre_reviews': review_count,
                'date_experience': date_of_experience,
                'reply': None
            }

        # Ajouter les données du commentaire (avec ou sans réponse) à la liste des commentaires de l'agence
        commentaires_par_agence.append(comment_data)
        i += 1

    driver.close()
    driver.switch_to.window(driver.window_handles[0])


# Enregistrement des données des commentaires dans un fichier JSON
with open(r'C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\commentaires.json', 'w', encoding='utf-8') as f:
    json.dump(commentaires_par_agence, f, ensure_ascii=False, indent=4)
print("Les données des commentaires ont été enregistrées dans le fichier commentaires.json.")

driver.quit()


Bouton cliqué avec succès.
Les données des commentaires ont été enregistrées dans le fichier commentaires.json.
